In [1]:
import pandas as pd
import plotly.express as px
from extra_structs import categories_translations, special_characters
from helpers import load_data_as_df, check_missing_values, time_based_train_test_split
import folium
from folium.plugins import MarkerCluster
import gc

# Customers

Let's start with the basic stats about the customers.


In [ ]:
customers_df = load_data_as_df('customers')
assert customers_df.customer_id.nunique() == len(customers_df)
assert customers_df.customer_unique_id.nunique() <= len(customers_df)
customers_df.head()

In [ ]:
# check if customer_city column contains any special characters that should not be expected in a city name
customers_df[~customers_df.customer_city.str.contains(r'^[a-zA-Z\s\'-]+$')]


In [ ]:
check_missing_values(customers_df)

In [ ]:
total_customers = customers_df.customer_id.nunique()
total_uq_customers = customers_df.customer_unique_id.nunique()

print(f'Total customers: {total_customers}')
print(f'Total unique customers: {total_uq_customers}')

In [ ]:
# How many customers are there in each state?
customer_states = pd.DataFrame(customers_df.customer_state.value_counts(normalize=True))

# add to customer_states a cumsum column so that we can see how many states are required for good coverage
customer_states['cumsum'] = customer_states.cumsum()
customer_states.head(6)

In [ ]:
# How many customers are there in each state?
customer_cities = pd.DataFrame(customers_df.customer_city.value_counts(normalize=True))
# add to customer_states a cumsum column so that we can see how many states are required for good coverage
customer_cities['cumsum'] = customer_cities.cumsum()
customer_cities.head(10)

# Geolocation

We could merge the dataframes and visualize the customers on a map.

In [ ]:
geolocation_df = load_data_as_df('geolocation')
geolocation_df.sort_values(
    by=['geolocation_zip_code_prefix', 'geolocation_lat', 'geolocation_lng'],
    ascending=False).head(4)

In [ ]:
check_missing_values(geolocation_df)

In [ ]:
geolocation_df.geolocation_city.value_counts().head(5)

## Minutes

Before merging with the customer data, there are at least 2 questions to address:
1. The data is very large, and each postal code is translated to different lat, lon locations.
2. City names are not standardized, e.g.:
    - "São Paulo" vs "Sao Paulo" 
    - "sao joao do pau d%26apos%3balho" vs "sao joao do pau dbalho"

In [ ]:
# Create a function to replace special characters
def replace_special_characters(city_name):
    for special_char, replacement in special_characters.items():
        city_name = city_name.replace(special_char, replacement)
    return city_name

geolocation_df['geolocation_city'] = geolocation_df['geolocation_city'].str.replace(r'[^a-zA-Z\s]', '', regex=True)
geolocation_df['geolocation_city'] = geolocation_df['geolocation_city'].apply(replace_special_characters)

geolocation_clean_df = geolocation_df.groupby(['geolocation_zip_code_prefix', 'geolocation_city']).agg(
    geolocation_lat=('geolocation_lat', 'mean'),
    geolocation_lng=('geolocation_lng', 'mean')
).reset_index()

del geolocation_df
gc.collect()

In [ ]:
geolocation_clean_df[geolocation_clean_df['geolocation_zip_code_prefix'] == 17970]

We finish by removing duplicates for each `customer_zip_code_prefix` .
As can be seen above, it is due to the fact that the city name is not standardized.


In [ ]:
original_len = len(geolocation_clean_df)
geolocation_clean_df = geolocation_clean_df.drop_duplicates(subset=['geolocation_zip_code_prefix'])
diff = original_len - len(geolocation_clean_df)
percent = round(diff / original_len * 100, 2)
print(f'Removed {diff} ({percent}%) duplicates')

31% is a pretty high percentage, due to lack of time I will not be able to address this.  
This issue should be looked if there's time.

# Map Visualization

We will now visualize the different locations on a map.  
Since there are 15K data points, we will use a marker cluster and only a random sample of 10%.


In [ ]:
random_sample = geolocation_clean_df.sample(frac=0.1)

# Create a base map
map = folium.Map(location=[-23.5505, -46.6333], zoom_start=12)

# Add a marker cluster to the map
marker_cluster = MarkerCluster().add_to(map)
errors = []
error_count = 0
# Add markers to the map based on the random_sample
for index, row in random_sample.iterrows():
    try:
        folium.Marker(
            location=[row['geolocation_lat'], row['geolocation_lng']],
        ).add_to(marker_cluster)
    except:
        errors.append(index)
        error_count += 1

map

In [ ]:
# How many errors did we get?
error_pct = round(error_count / len(random_sample) * 100, 4)
print(f'Error rate: {error_pct}%')

# Order Items

We now inspect the items data

In [ ]:
order_items_df = load_data_as_df('order_items')
order_items_df.head()

In [ ]:
uq_products = order_items_df.product_id.nunique()
uq_sellers = order_items_df.seller_id.nunique()

print(f'Unique products: {uq_products}')
print(f'Unique sellers: {uq_sellers}')

In [ ]:
# check how many products per seller
order_items_df.groupby('seller_id').product_id.nunique().describe()

In [ ]:
fig = px.histogram(order_items_df, x="freight_value", marginal="box") 
fig.show()

In [ ]:
fig = px.histogram(order_items_df, x="price", marginal="box") 
fig.show()

In [ ]:
order_items_df['freight_component'] = order_items_df.freight_value / order_items_df.price
order_items_df.freight_component.describe()

It would be interesting to see if for some products the *freight price* is actually higher than the product price itself

In [ ]:
msk = order_items_df.freight_component > 1
order_items_df[msk].freight_component.describe()

**Freight Component**

We see that indeed in some cases the freight price is greater than the price of the item. 
As a sanity check, we should sample and check some of these items with SME.

# Orders Dataset

Now we inspect the `Orders` dataset.

This dataset as a lot of temporal features, and it should show us the issue that the product manager talked about - delivery delays.

While we will perform this analysis in the next notebook, we will currently just add the time delta features and get an overview of the order cycle times.

In [ ]:
orders_df = load_data_as_df('orders')
orders_df['order_approved_at'] = pd.to_datetime(orders_df['order_approved_at'])
orders_df['order_delivered_carrier_date'] = pd.to_datetime(orders_df['order_delivered_carrier_date'])
orders_df['order_delivered_customer_date'] = pd.to_datetime(orders_df['order_delivered_customer_date'])
orders_df['order_estimated_delivery_date'] = pd.to_datetime(orders_df['order_estimated_delivery_date'])
orders_df['order_purchase_timestamp'] = pd.to_datetime(orders_df['order_purchase_timestamp'])
orders_df.head()


In [ ]:
check_missing_values(orders_df)

Missing value in the dates data makes sense if the order lifecycle was not completed. 
However, just to make sure the NAs correspond to the correct logic, we will double-check the order status for these orders.

In [ ]:
orders_df[orders_df.isna().any(axis=1)].order_status.value_counts(normalize=True)

In [ ]:
orders_df[orders_df.isna().any(axis=1) & (orders_df.order_status == 'shipped')]

It looks like some of the orders were shipped a very long time ago.
These orders may have been delievered, but the data was not updated.
Or they were never delievered, in this case it would be interesting to know why.

In [ ]:
orders_df.order_status.value_counts(normalize=True)

**Order Status**

Overall we see pretty good stats here, the cancellation rate is extremely low for this kind of platform.

In [ ]:
# Get the orders time range, based on order_purchase_timestamp
first_order = orders_df.order_purchase_timestamp.min()
last_order = orders_df.order_purchase_timestamp.max()
print(f"Orders time range: {first_order} - {last_order}")

In [ ]:
# plot orders over time

# Group by month and count the number of orders
orders_per_month = orders_df.resample('ME', on='order_purchase_timestamp').size().reset_index(name='orders_count')

# Create a line plot


fig = px.line(orders_per_month, x='order_purchase_timestamp', y='orders_count', 
              title='Number of Orders per Month', 
              labels={'order_purchase_timestamp': 'Month', 'orders_count': 'Number of Orders'})

fig.show()


### Business Growth

The number of orders per month has grown pretty well during the first year since launch.  
Starting 2018, the number of orders reached a plateau and has been pretty stable since then.
Finally, it looks like Sep 2018 data was not updated just yet.

It would be interesting to talk with the product manager to understand the product strategy and see if it aligns with the data.

In [31]:
assert orders_df.order_id.nunique() == len(orders_df)

In [32]:
# Adding time delta features: 
#  - Time between order_purchase_timestamp and order_approved_at (approval time)
#  - Time between order_approved_at and order_delivered_carrier_date delivery
#  - Time between order_delivered_carrier_date and order_delivered_customer_date
#  - Time between order_approved_at and order_delivered_customer_date (our target variable)
# Results are in days

seconds_in_day = 60 * 60 * 24
def add_time_delta_features(df: pd.DataFrame, denominator: int = seconds_in_day):
    df['approval_time'] = (df['order_approved_at'] - df['order_purchase_timestamp']).dt.total_seconds() / denominator
    df['approved_to_carrier'] = (df['order_delivered_carrier_date'] - df['order_approved_at']).dt.total_seconds() / denominator
    df['carrier_to_customer'] = (df['order_delivered_customer_date'] - df['order_delivered_carrier_date']).dt.total_seconds() / denominator
    df['approval_to_customer'] = (df['order_delivered_customer_date'] - df['order_approved_at']).dt.total_seconds() / denominator
    return df

orders_df = add_time_delta_features(orders_df)

In [ ]:
# Validation checks:
#   all the time deltas should be non-negative
def get_negative_time_deltas(df: pd.DataFrame):
    data_anomalies = pd.DataFrame()
    for col in ['approval_time', 'approved_to_carrier', 'carrier_to_customer', 'approval_to_customer']:
        mask = df[col] < 0
        if mask.any():
            print(f"{col} has {mask.sum()} negative values")
            data_anomalies = pd.concat([data_anomalies, df[mask]])

    return data_anomalies

delta_anomalies = get_negative_time_deltas(orders_df)
if len(delta_anomalies) > 0:
    delta_anomalies.sample(6)

In [ ]:
delta_time_anomlies = round(len(delta_anomalies) / len(orders_df) * 100, 2)
print(f"Percentage of time delta anomalies: {delta_time_anomlies}%")

Some dates could be wrong, because they don't make sense with the order lifecycle.  
Anyway, it is less than 1.5% of the data, which is not negligible, but seems manageable.

We need to look into these cases more closely in the future and see if we can find the root cause.

In [ ]:
# remove all the anomalies from the original `orders` dataframe
orders = orders_df[~orders_df.index.isin(delta_anomalies.index)]
del delta_anomalies
gc.collect()    

In [ ]:
# plot a boxplot for each of the time delta features
fig = px.box(orders, y=['approval_time',
                        'approved_to_carrier',
                        'carrier_to_customer',
                        'approval_to_customer'],
                        title='Order Lifecycle Time Deltas',
                        labels={'value': 'Time In Days'})
fig.show()



In [ ]:
fig = px.box(orders, x='approval_to_customer',
             title='Purchased to Delieverd Time Deltas',
             labels={'value': 'Time In Days'})
fig.show()



# Products Dataset

Now we take a look at the contents that are traded in the marketplace.

In [ ]:
products_df = load_data_as_df('products')
products_df.head()

In [39]:
assert products_df.product_id.nunique() == len(products_df), "duplicated product IDs"

In [ ]:
check_missing_values(products_df)

In [ ]:
products_df.product_category_name.value_counts(normalize=True)

In [ ]:
# replace the product_category_name with the English translation
products_df = products_df.merge(categories_translations, how='left')
products_df.drop(columns=['product_category_name'], inplace=True)
products_df.head()


In [ ]:
uncategorized = len(products_df[pd.isna(products_df.product_category)])
uncategoried_ratio = round(uncategorized / len(products_df) * 100, 2)
print(f"Percentage of uncategorized products: {uncategoried_ratio}%")


In general, having uncategorized products should be discouraged.  
We can try to correcly categorize the uncategorized products, but since the percentage is low, we will ignore this for now.



In [44]:
products_df['total_size'] = products_df.product_length_cm * products_df.product_height_cm * products_df.product_width_cm

In [45]:
assert order_items_df.product_id.nunique() == products_df.product_id.nunique(), "Mismatch in product_id between order_items and products_df"


In [ ]:
# Visualize the total size of the products
fig = px.histogram(products_df, x='total_size', title='Total Size Histogram')
fig.show()

In [ ]:
products_df.product_category.value_counts(normalize=True).head(10).cumsum()

In [ ]:
# visualize product weight
fig = px.histogram(products_df, x='product_weight_g', title='Product Weight Histogram')
fig.show()

# Sellers Dataset

Finally, we take a look at the sellers.


In [ ]:
sellers_df = load_data_as_df('sellers')
sellers_df.head()

In [50]:
assert sellers_df.seller_id.nunique() == len(sellers_df)

In [ ]:
check_missing_values(sellers_df)

In [ ]:
sellers_df.seller_city.value_counts(normalize=True).cumsum().head(20)

In [ ]:
sellers_df.seller_city.nunique()

# Merging the DataFrames

Now we merge all the dataframes to provide the data for EDA and Modelling.

The merge will be as follows:

 - `Sellers`, `Geo_location` by `zip_code_prefix`
 - `Customers`, `Geo_location` by `zip_code_prefix`
 - `Order_items`, `products` by `product_id`
 - `Orders`, `order_items` by `order_id`
 - `Orders`, `sellers` by `seller_id`
 - `Orders`, `customers` by `customer_id`

Everntaually, our `Orders` dataframe will be the main dataframe that will contain all the information. 
Do note, that even though we will perform *left* joins to `Orders`, the number of rows  **will increase** since a single order could contain multiple items and sellers.




In [54]:
sellers_extended_df = sellers_df.merge(geolocation_clean_df, how='left', 
                                       left_on='seller_zip_code_prefix',
                                       right_on='geolocation_zip_code_prefix')

In [55]:
# rename geolocation_ prefix to _customer prefix in all relevant columns
sellers_extended_df = sellers_extended_df.rename(columns={
    'geolocation_lat': 'seller_lat',
    'geolocation_lng': 'seller_lng',
})

sellers_extended_df.drop(columns=["geolocation_zip_code_prefix", "geolocation_city"], inplace=True)



In [56]:
customers_extended_df = customers_df.merge(geolocation_clean_df, how='left', 
                                       left_on='customer_zip_code_prefix',
                                       right_on='geolocation_zip_code_prefix')

In [57]:
# rename geolocation_ prefix to _customer prefix in all relevant columns
customers_extended_df = customers_extended_df.rename(columns={
    'geolocation_lat': 'customer_lat',
    'geolocation_lng': 'customer_lng',
})

customers_extended_df.drop(columns=["geolocation_zip_code_prefix", "geolocation_city"], inplace=True)



In [58]:
order_items_extended_df = order_items_df.merge(products_df, how='left', on='product_id')

In [ ]:
del sellers_df
del customers_df
del geolocation_clean_df
del order_items_df
gc.collect()

In [60]:
orders_df_extended = orders_df.merge(order_items_extended_df, how='left', on='order_id')

In [61]:
orders_extended_df = orders_df_extended.merge(sellers_extended_df, how='left', on='seller_id')

In [62]:
orders_extended_df = orders_extended_df.merge(customers_extended_df, how='left', on='customer_id')

In [63]:
# Remove redundant columns like the seller_zip_code_prefix, ustomer_zip_code_prefix and geolocation_city
orders_extended_df.drop(columns=[col for col in orders_extended_df.columns if col.endswith('_prefix')], inplace=True)

In [ ]:
orders_extended_df.sample(6)

# Divide the data into training and test sets

Now, we want to dive deeper into the EDA.  
In order to make the test set close to the real world scenario, we will use the latest data as the test set. 

Also, since there are only 2 years of data, yearly seasonality will not be considered for modelling (but weekly / monthly seasonality will be considered)

In [ ]:
# Split the data - most recent 20% as test set
train_df, test_df = time_based_train_test_split(orders_extended_df, 'order_purchase_timestamp', test_size=0.2)

print(f"Train set: {len(train_df)} samples ({len(train_df)/len(orders_extended_df):.1%})")
print(f"Test set: {len(test_df)} samples ({len(test_df)/len(orders_extended_df):.1%})")

# Verify the split by checking date ranges
print(f"Train date range: {train_df['order_purchase_timestamp'].min()} to {train_df['order_purchase_timestamp'].max()}")
print(f"Test date range: {test_df['order_purchase_timestamp'].min()} to {test_df['order_purchase_timestamp'].max()}")

In [67]:
train_df.to_csv('../data/train_df.csv', index=False)
test_df.to_csv('../data/test_df.csv', index=False)